## 📊 SQL Window Functions & Pandas Equivalents

| #  | SQL Function | Equivalent in Python (pandas) | Description |
|----|-------------|-------------------------------|-------------|
| 1  | `ROW_NUMBER() OVER (PARTITION BY ... ORDER BY ...)` | `df['rank'] = df.groupby(['partition_column']).cumcount() + 1` | Assigns a sequential number to each row within the partition. |
| 2  | `RANK() OVER (PARTITION BY ... ORDER BY ...)` | `df['rank'] = df.groupby(['partition_column'])['order_column'].rank(method='min')` | Ranks rows with ties, skipping numbers when duplicates exist. |
| 3  | `DENSE_RANK() OVER (PARTITION BY ... ORDER BY ...)` | `df['dense_rank'] = df.groupby(['partition_column'])['order_column'].rank(method='dense')` | Ranks rows without skipping numbers, even with ties. |
| 4  | `NTILE(n) OVER (PARTITION BY ... ORDER BY ...)` | `df['ntile'] = pd.qcut(df['order_column'], q=n, labels=False) + 1` | Divides rows into 'n' approximately equal groups. |
| 5  | `SUM(col) OVER (PARTITION BY ... ORDER BY ...)` | `df['sum'] = df.groupby(['partition_column'])['column'].cumsum()` | Cumulative sum of values within the partition. |
| 6  | `AVG(col) OVER (PARTITION BY ... ORDER BY ...)` | `df['avg'] = df.groupby(['partition_column'])['column'].expanding().mean().reset_index(level=0, drop=True)` | Cumulative average of values within the partition. |
| 7  | `MIN(col) OVER (PARTITION BY ... ORDER BY ...)` | `df['min'] = df.groupby(['partition_column'])['column'].cummin()` | Minimum value in the partition up to the current row. |
| 8  | `MAX(col) OVER (PARTITION BY ... ORDER BY ...)` | `df['max'] = df.groupby(['partition_column'])['column'].cummax()` | Maximum value in the partition up to the current row. |
| 9  | `COUNT(col) OVER (PARTITION BY ... ORDER BY ...)` | `df['count'] = df.groupby(['partition_column']).cumcount() + 1` | Counts the number of rows within the partition. |
| 10 | `LAG(col, 1) OVER (PARTITION BY ... ORDER BY ...)` | `df['lag'] = df.groupby(['partition_column'])['column'].shift(1)` | Returns the previous row’s value within the partition. |
| 11 | `LEAD(col, 1) OVER (PARTITION BY ... ORDER BY ...)` | `df['lead'] = df.groupby(['partition_column'])['column'].shift(-1)` | Returns the next row’s value within the partition. |
| 12 | `FIRST_VALUE(col) OVER (PARTITION BY ... ORDER BY ...)` | `df['first_value'] = df.groupby(['partition_column'])['column'].transform('first')` | Returns the first value in the partition. |
| 13 | `LAST_VALUE(col) OVER (PARTITION BY ... ORDER BY ...)` | `df['last_value'] = df.groupby(['partition_column'])['column'].transform('last')` | Returns the last value in the partition. |
| 14 | `NTH_VALUE(col, n) OVER (PARTITION BY ... ORDER BY ...)` | `df['nth_value'] = df.groupby(['partition_column'])['column'].transform(lambda x: x.iloc[n-1] if len(x) >= n else None)` | Returns the nth value in the partition. |


In [ ]:
import pandas as pd
import pandasql as ps

In [4]:
df = pd.read_csv('./csv/SuperStoreOrders.csv')

df.head()

,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,...,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year
0,AG-2011-2040,1/1/2011,6/1/2011,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,Africa,...,Office Supplies,Storage,"Tenex Lockers, Blue",408,2,0.0,106.140,35.46,Medium,2011
1,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Office Supplies,Supplies,"Acme Trimmer, High Speed",120,3,0.1,36.036,9.72,Medium,2011
2,HU-2011-1220,1/1/2011,5/1/2011,Second Class,Annie Thurman,Consumer,Budapest,Hungary,EMEA,EMEA,...,Office Supplies,Storage,"Tenex Box, Single Width",66,4,0.0,29.640,8.17,High,2011
3,IT-2011-3647632,1/1/2011,5/1/2011,Second Class,Eugene Moren,Home Office,Stockholm,Sweden,EU,North,...,Office Supplies,Paper,"Enermax Note Cards, Premium",45,3,0.5,-26.055,4.82,High,2011
4,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",114,5,0.1,37.770,4.70,Medium,2011


### 1

In [6]:
# Criando uma consulta SQL com RANK() OVER (PARTITION BY ...)
query = """
    SELECT 
        order_id,
        category,
        sales,
        ROW_NUMBER() OVER (PARTITION BY category ORDER BY sales DESC) AS sales_rank
    FROM df
"""

# Executando a consulta SQL com pandasql
df_row_number = ps.sqldf(query, locals())

df_row_number

,order_id,category,sales,sales_rank
0,IN-2013-11448,Furniture,998,1
1,US-2013-117772,Furniture,996,2
2,ES-2014-1015287,Furniture,996,3
3,ES-2012-5645358,Furniture,994,4
4,IV-2011-6850,Furniture,993,5
...,...,...,...,...
51285,CA-2012-135685,Technology,"1,000",10137
51286,CA-2012-156104,Technology,"1,000",10138
51287,CA-2013-166429,Technology,"1,000",10139
51288,IN-2014-37033,Technology,"1,000",10140
